In [1]:
import json
import numpy as np
import redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from sentence_transformers import SentenceTransformer

In [15]:
client = redis.Redis(host="localhost", port=6379, decode_responses=True, password='password')

In [ ]:
# 임베딩 모델 Load
model_path = "/Users/kim-chulsoo/Desktop/PROJECT/LLMOps/models/embedding_model/fine_tune_model_lora_merge_ver"
embedder = SentenceTransformer(model_path)

In [304]:
temp = [
    {
        "chat_id" : 'a123123',
        "chat": "올해 매출액이 얼마지?",
        "used_tool": "smarta_tool_A",
        "answer": "매출액 얼마인지 모름요",
        "combine_chat": "Q.올해 매출액이 얼마지?  A.매출액 얼마인지 모름요",
        "tags": 'tag1, ta2, tag3',
        "state": 1 # 0.미사용, 1.사용중
    },
    {
        "chat_id" : 'a123124',
        "chat": "부가세 신고기간이 언제지??",
        "used_tool": "smarta_tool_B",
        "answer": "2024년 부가세 1기 예정신고는 2024년 4월 25일이 마감일입니다.",
        "combine_chat": "Q.부가세 신고기간이 언제지??  A.2024년 부가세 1기 예정신고는 2024년 4월 25일이 마감일입니다.",
        "tags": 'tag5, tag6',
        "state": 1 # 0.미사용, 1.사용중
    },
    {
        "chat_id" : 'a123126',
        "chat": "더존비즈온의 50세 이상 직원은?",
        "used_tool": "smarta_tool_K",
        "answer": "더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...",
        "combine_chat": "Q.더존비즈온의 50세 이상 직원은?  A.더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...",
        "tags": 'tag8, tag10',
        "state": 1 # 0.미사용, 1.사용중
    },
    {
        "chat_id" : 'a123128',
        "chat": "더존비즈온의 60세 이상 직원은?",
        "used_tool": "smarta_tool_K",
        "answer": "더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...",
        "combine_chat": "Q.더존비즈온의 60세 이상 직원은?  A.더존비즈온의 60세 이상직원은 총 3명입니다. \n 1) 카리나\n 2) 윈터\n 3) 차은우",
        "tags": 'tag5, tag99',
        "state": 0 # 0.미사용, 1.사용중
    },
    {
        "chat_id" : 'a123129',
        "chat": "나이가 환갑이 넘은 직원은?",
        "used_tool": "smarta_tool_K",
        "answer": "올해 예순(환갑)이 넘은 직원 목록은 아래와 같습니다. \n 1) 박지성\n 2) 차범근\n 3) 손웅정 ...",
        "combine_chat": "Q.나이가 환갑이 넘은 직원은?  A.올해 예순(환갑)이 넘은 직원 목록은 아래와 같습니다. \n 1) 박지성\n 2) 차범근\n 3) 손웅정 ...",
        "tags": 'tag5, tag88',
        "state": 1 # 0.미사용, 1.사용중
    },
    {
        "chat_id" : 'a123130',
        "chat": "더존비즈온의 부가세 예정신고 금액은?",
        "used_tool": "smarta_tool_K",
        "answer": "더존비즈온의 부가세 예정신고 금액은 아래와 같습니다. \n 더존비즈온 직원급여와 포함한 예정신고금액은 100만원",
        "combine_chat": "더존비즈온의 부가세 예정신고 금액과 직원급여 지급액은?  A.더존비즈온의 부가세 예정신고 금액은 아래와 같습니다. \n 예정신고금액은 100만원이며, 직원 급여금액 총액은 10만원입니다",
        "tags": 'tag5, tag78',
        "state": 1 # 0.미사용, 1.사용중
    },
]

## 1) Redis 데이터 저장(일반데이터) 및 조회

In [181]:
pipeline = client.pipeline()
for i, sample in enumerate(temp):
    redis_key = f"chat:{i}"
    pipeline.json().set(redis_key, "$", sample)
res = pipeline.execute()

In [182]:
client.json().get("chat:1", "$.chat")

['부가세 신고기간이 언제지??']

In [183]:
client.json().get("chat:2", "$.combine_chat")

['Q.더존비즈온의 50세 이상 직원은?  A.더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...']

In [184]:
client.json().get("chat:2", "$.chat_id")

['a123126']

## 2) 벡터 검색
### 1. 텍스트 임베딩 생성

In [185]:
keys = sorted(client.keys("chat:*"))
keys

['chat:0', 'chat:1', 'chat:2', 'chat:3']

In [186]:
descriptions = client.json().mget(keys, "$.combine_chat")
descriptions = [item for sublist in descriptions for item in sublist]
descriptions

['Q.올해 매출액이 얼마지?  A.매출액 얼마인지 모름요',
 'Q.부가세 신고기간이 언제지??  A.2024년 부가세 1기 예정신고는 2024년 4월 25일이 마감일입니다.',
 'Q.더존비즈온의 50세 이상 직원은?  A.더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
 'Q.더존비즈온의 60세 이상 직원은?  A.더존비즈온의 60세 이상직원은 총 3명입니다. \n 1) 카리나\n 2) 윈터\n 3) 차은우']

In [187]:
# json.mget을 통해 'combine_chat'에 해당하는 데이터를 목록으로 수집함[[combine_chat1], [combine_chat2]...] 구조
descriptions = client.json().mget(keys, "$.combine_chat")
# [combine_chat1, combine_chat2...] 구조로 하나의 목록 내부에 데이터를 적재
descriptions = [item for sublist in descriptions for item in sublist]
# 임베딩 값 추출
embeddings = embedder.encode(descriptions).astype(np.float32).tolist()
VECTOR_DIMENSION = len(embeddings[0])
VECTOR_DIMENSION

768

In [188]:
# 저장된 데이터들의 key별로 chat데이터에 대한 임베딩 값 저장
for key, embedding in zip(keys, embeddings):
    pipeline.json().set(key, "$.chat_embeddings", embedding)
pipeline.execute()

[True, True, True, True]

In [189]:
res = client.json().get("chat:1")
res

{'chat_id': 'a123124',
 'chat': '부가세 신고기간이 언제지??',
 'used_tool': 'smarta_tool_B',
 'answer': '2024년 부가세 1기 예정신고는 2024년 4월 25일이 마감일입니다.',
 'combine_chat': 'Q.부가세 신고기간이 언제지??  A.2024년 부가세 1기 예정신고는 2024년 4월 25일이 마감일입니다.',
 'tags': 'tag5, tag6',
 'state': 1,
 'chat_embeddings': [0.10754521936178207,
  0.12482050061225893,
  0.5994256734848022,
  -0.4855499267578125,
  -0.028293531388044357,
  -0.4610951542854309,
  -0.6846954822540283,
  -0.039320409297943115,
  -0.03880023956298828,
  0.2220822423696518,
  0.02637998014688492,
  -0.6517055630683899,
  -0.3855478763580322,
  -0.2813422679901123,
  -0.0951172485947609,
  0.14488939940929413,
  0.07076884806156158,
  0.28118735551834106,
  -0.09229877591133118,
  0.4165066182613373,
  -0.35574185848236084,
  -0.2390435636043549,
  0.27813538908958435,
  0.29535654187202454,
  0.21846657991409305,
  -0.210820734500885,
  -0.1958571821451187,
  0.49337098002433777,
  -0.26981037855148315,
  -0.18693622946739197,
  0.03708776831626892,
  0.8523

### 2. 벡터 필드 정의하고 인덱스 생성하기 (RediSearch에만 있음)

In [190]:
schema = (
    # no_stem : 스테밍(stemming)옵션은 검색 시, 단어의 어근(stem)을 추출해서 검색결과의 확장성을 높이는 옵션이다. 
    # 예를들어, '안녕하세요'에서 '안녕'이라는 어근을 추출해서 어근 단위로 검색하여 더 많은 검색결과를 가져오는 방식이다.
    TextField("$.chat_id", no_stem=False, as_name="chat_id"),
    TextField("$.chat", no_stem=True, as_name="chat"),
    TextField("$.used_tool", no_stem=False, as_name="used_tool"),
    TextField("$.answer", no_stem=True, as_name="answer"),
    TextField("$.combine_chat", no_stem=True, as_name="combine_chat"),
    TagField("$.tags", as_name="tags"),
    NumericField("$.state", as_name="state"),
    # FLAT는 전체 항목에 대한 벡터검색(Brute-force) : 정확 + 느림 / HNSW - 대략적 검색, 빠른결과, 낮은 정확도
    # DISTANCE_METRIC : cosine, internal product, euclidean distance
    VectorField(
        "$.chat_embeddings",
        "FLAT",
        {
            "TYPE": "FLOAT32",
            "DIM": VECTOR_DIMENSION,
            "DISTANCE_METRIC": "COSINE",
        },
        as_name="vector",
    ),
)
definition = IndexDefinition(prefix=["chat:"], index_type=IndexType.JSON)
res = client.ft("idx:chat_vec").create_index(fields=schema, definition=definition)
res

'OK'

### 3. 인덱스 상태 확인

In [191]:
info = client.ft("idx:chat_vec").info()
num_docs = info["num_docs"]
indexing_failures = info["hash_indexing_failures"]
print(f"{num_docs} documents indexed with {indexing_failures} failures")

4 documents indexed with 0 failures


### 4. 벡터 검색하기

In [192]:
# 질문 임베딩 값 추출
queries = [
    "더존비즈온의 30세이상 직원은?"
    ]
encoded_queries = embedder.encode(queries)
len(encoded_queries)

1

In [233]:
def create_query_table(query, queries, encoded_queries, extra_params=None):
    """
    Creates a query table.
    """
    results_list = []
    for i, encoded_query in enumerate(encoded_queries):
        result_docs = (
            client.ft("idx:chat_vec")
            .search(
                query,
                {"query_vector": np.array(encoded_query, dtype=np.float32).tobytes()}
                | (extra_params if extra_params else {}),
            )
            .docs
        )
        for doc in result_docs:
            vector_score = round(1 - float(doc.vector_score), 2)
            results_list.append(
                {
                    "query": queries[i],
                    "score": vector_score,
                    "id": doc.id,
                    "chat_id": doc.chat_id,
                    "combine_chat": doc.combine_chat,
                    "tags": doc.tags,
                    "used_tool": doc.used_tool
                }
            )

    # Optional: convert the table to Markdown using Pandas
    return results_list

In [229]:
query = (
    Query("(*)=>[KNN 2 @vector $query_vector AS vector_score]")
    .sort_by("vector_score")
    .return_fields("vector_score", "chat_id", "combine_chat", "tags", "used_tool")
    .dialect(2)
)

table = create_query_table(query, queries, encoded_queries)
table

[{'query': '더존비즈온의 30세이상 직원은?',
  'score': 0.58,
  'id': 'chat:2',
  'chat_id': 'a123126',
  'combine_chat': 'Q.더존비즈온의 50세 이상 직원은?  A.더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
  'tags': 'tag8, tag10',
  'used_tool': 'smarta_tool_K'},
 {'query': '더존비즈온의 30세이상 직원은?',
  'score': 0.56,
  'id': 'chat:3',
  'chat_id': 'a123128',
  'combine_chat': 'Q.더존비즈온의 60세 이상 직원은?  A.더존비즈온의 60세 이상직원은 총 3명입니다. \n 1) 카리나\n 2) 윈터\n 3) 차은우',
  'tags': 'tag5, tag99',
  'used_tool': 'smarta_tool_K'}]

## 3) TAG CRUD
### 1. TAG 추가

In [195]:
key = "chat:1"
add_tag = 'chat999'
origin_tags = client.json().get(key, "$.tags")[0]
print(f'현재 태그 목록 : {origin_tags}')
new_tag_list = origin_tags.split(", ")
new_tag_list.append(add_tag)
new_tags = ", ".join(new_tag_list)
client.json().set(key, "$.tags", new_tags)
new_tag_list = client.json().get(key, "$.tags")[0]
print(f'추가완료 후 태그 목록 : {new_tag_list}')

현재 태그 목록 : tag5, tag6
추가완료 후 태그 목록 : tag5, tag6, chat999


### 2. TAG 삭제

In [196]:
delete_tag = 'chat999'
origin_tags = client.json().get(key, "$.tags")[0]
tag_list = origin_tags.split(", ")

update_tag_list = [tag for tag in tag_list if tag!=delete_tag]
update_tags = ", ".join(update_tag_list)
client.json().set(key, "$.tags", update_tags)
print(f'삭제완료 후 태그 목록 : {client.json().get(key, "$.tags")[0]}')

삭제완료 후 태그 목록 : tag5, tag6


### 3. TAG 검색

In [210]:
search_tag = 'tag5'
search_query = f"@tags:{{{search_tag}}}" 
# '@tags:{tag5} 와 같이 검색대상을 중괄호로 감싸야 검색이 가능하다.
results = client.ft('idx:chat_vec').search(search_query)

# 결과 출력
print(f'Search Results:')
for doc in results.docs:
    doc_data = json.loads(doc.json)
    print('- chat_id : ', doc_data['chat_id'])
    print('- chat : ', doc_data['chat'])
    print('- answer : ', doc_data['answer'])
    print('- tags : ', doc_data['tags'])
    print('- used_tool : ', doc_data['used_tool'])
    print('- state : ', doc_data['state'])
    print('\n')

Search Results:
- chat_id :  a123128
- chat :  더존비즈온의 60세 이상 직원은?
- answer :  더존비즈온의 50세 이상직원은 총 5명입니다. 
 1) 이순신
 2) 세종대왕
 3) 정약용 ...
- tags :  tag5, tag99
- used_tool :  smarta_tool_K
- state :  0


- chat_id :  a123124
- chat :  부가세 신고기간이 언제지??
- answer :  2024년 부가세 1기 예정신고는 2024년 4월 25일이 마감일입니다.
- tags :  tag5, tag6
- used_tool :  smarta_tool_B
- state :  1




## 4) 케이스별 조건 검색
### Case1. State 조건을 포함한 Tag데이터 검색

In [212]:
search_tag = 'tag5'
search_query = f"@state:[1 1] @tags:{{{search_tag}}}" 
# '@tags:{tag5} 와 같이 검색대상을 중괄호로 감싸야 검색이 가능하다.
results = client.ft('idx:chat_vec').search(search_query)

# 결과 출력
print(f'Search Results:')
for doc in results.docs:
    doc_data = json.loads(doc.json)
    print('- chat_id : ', doc_data['chat_id'])
    print('- chat : ', doc_data['chat'])
    print('- answer : ', doc_data['answer'])
    print('- tags : ', doc_data['tags'])
    print('- used_tool : ', doc_data['used_tool'])
    print('- state : ', doc_data['state'])
    print('\n')

Search Results:
- chat_id :  a123124
- chat :  부가세 신고기간이 언제지??
- answer :  2024년 부가세 1기 예정신고는 2024년 4월 25일이 마감일입니다.
- tags :  tag5, tag6
- used_tool :  smarta_tool_B
- state :  1




### Case2. State값이 1인 데이터중에서 Vector가 가장 유사한 데이터 2개 추출

In [288]:
# 질문 임베딩 값 추출
queries = [
    "더존비즈온의 30세이상 직원은?"
    ]
encoded_queries = embedder.encode(queries)

In [234]:
query = (
    Query("@state:[1 1]=>[KNN 3 @vector $query_vector AS vector_score]")
    .sort_by("vector_score")
    .return_fields("vector_score", "chat_id", "combine_chat", "tags", "used_tool")
    .dialect(2)
)

table = create_query_table(query, queries, encoded_queries)
table

[{'query': '더존비즈온의 30세이상 직원은?',
  'score': 0.58,
  'id': 'chat:2',
  'chat_id': 'a123126',
  'combine_chat': 'Q.더존비즈온의 50세 이상 직원은?  A.더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
  'tags': 'tag8, tag10',
  'used_tool': 'smarta_tool_K'},
 {'query': '더존비즈온의 30세이상 직원은?',
  'score': 0.17,
  'id': 'chat:1',
  'chat_id': 'a123124',
  'combine_chat': 'Q.부가세 신고기간이 언제지??  A.2024년 부가세 1기 예정신고는 2024년 4월 25일이 마감일입니다.',
  'tags': 'tag5, tag6',
  'used_tool': 'smarta_tool_B'},
 {'query': '더존비즈온의 30세이상 직원은?',
  'score': 0.16,
  'id': 'chat:0',
  'chat_id': 'a123123',
  'combine_chat': 'Q.올해 매출액이 얼마지?  A.매출액 얼마인지 모름요',
  'tags': '["tag1","tag2","tag3"]',
  'used_tool': 'smarta_tool_A'}]

### Case3. Hybrid Search(Lexical + Semantic) 테스트

In [305]:
temp_data = [
    {
        **sample,
        "chat_embeddings":  embedder.encode(sample['combine_chat']).astype(np.float32).tobytes()
    } for sample in temp
]

In [306]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex

schema = IndexSchema.from_dict({
  "index": {
    "name": "chat-history-index",
    "prefix": "chat",
    "storage": "hash"
  },
  "fields": [
    { "name": "chat_id", "type": "text" },
    { "name": "chat", "type": "text" },
    { "name": "used_tool", "type": "text" },
    { "name": "answer", "type": "text" },
    { "name": "combine_chat", "type": "text" },
    { "name": "tags", "type": "tag" },
    {
        "name": "chat_embeddings",
        "type": "vector",
        "attrs": {
            "dims": VECTOR_DIMENSION,
            "distance_metric": "COSINE",
            "algorithm": "FLAT",
            "datatype": "FLOAT32"
        }
    }
  ]
})

index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

14:51:12 redisvl.index.index INFO   Index already exists, overwriting.


In [307]:
index.load(temp_data)

['chat:bf56816098c0490aaaa4656b6c617f19',
 'chat:90ef3848312e47079d67640877391341',
 'chat:e73eb1f85f7145a88cf358853eb59782',
 'chat:f65e11af0beb44e1a1f7a4803618f613',
 'chat:4ea921230b5e4f78894e80fd806c0418',
 'chat:4af0ce62b7bb4fe3b13ce862f9b7bd0e']

In [308]:
# Sample user query (can be changed for comparisons)
user_query = "더존비즈온의 30세이상 직원은?"

In [309]:
# list of stopwords to filter out noise from query string
stopwords = set([
    "a", "is", "the", "an", "and", "are", "as", "at", "be", "but", "by", "for",
    "if", "in", "into", "it", "no", "not", "of", "on", "or", "such", "that", "their",
    "then", "there", "these", "they", "this", "to", "was", "will", "with"
])

def tokenize_query(user_query: str) -> str:
    """Convert a raw user query to a redis full text query joined by ORs"""
    tokens = [token.strip().strip(",").lower() for token in user_query.split()]
    return " | ".join([token for token in tokens if token not in stopwords])

# Example
tokenize_query(user_query)

'더존비즈온의 | 30세이상 | 직원은?'

In [310]:
from redisvl.query import VectorQuery, FilterQuery
from redisvl.query.filter import Text
from redisvl.redis.utils import convert_bytes, make_dict

return_fields = ['chat_id', 'chat', 'answer', 'combine_chat', 'state', 'tags']

def make_vector_query(user_query: str, num_results: int, filters = None) -> VectorQuery:
    """Generate a Redis vector query given user query string."""
    vector = embedder.encode(user_query).astype(np.float32).tobytes()
    query = VectorQuery(
        vector=vector,
        vector_field_name="chat_embeddings",
        num_results=num_results,
        return_fields=return_fields
    )
    if filters:
        query.set_filter(filters)
    
    return query


def make_ft_query(text_field: str, user_query: str, num_results: int) -> FilterQuery:
    """Generate a Redis full-text query given a user query string."""
    return FilterQuery(
        filter_expression=f"~({Text(text_field) % tokenize_query(user_query)})",
        num_results=num_results,
        return_fields=return_fields,
        dialect=4,
    ).scorer("BM25").with_scores()

In [311]:
query = make_vector_query(user_query, num_results=4)
index.query(query)

[{'id': 'chat:e73eb1f85f7145a88cf358853eb59782',
  'vector_distance': '0.421030580997',
  'chat_id': 'a123126',
  'chat': '더존비즈온의 50세 이상 직원은?',
  'answer': '더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
  'combine_chat': 'Q.더존비즈온의 50세 이상 직원은?  A.더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
  'state': '1',
  'tags': 'tag8, tag10'},
 {'id': 'chat:f65e11af0beb44e1a1f7a4803618f613',
  'vector_distance': '0.437910437584',
  'chat_id': 'a123128',
  'chat': '더존비즈온의 60세 이상 직원은?',
  'answer': '더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
  'combine_chat': 'Q.더존비즈온의 60세 이상 직원은?  A.더존비즈온의 60세 이상직원은 총 3명입니다. \n 1) 카리나\n 2) 윈터\n 3) 차은우',
  'state': '0',
  'tags': 'tag5, tag99'},
 {'id': 'chat:4ea921230b5e4f78894e80fd806c0418',
  'vector_distance': '0.562243103981',
  'chat_id': 'a123129',
  'chat': '나이가 환갑이 넘은 직원은?',
  'answer': '올해 예순(환갑)이 넘은 직원 목록은 아래와 같습니다. \n 1) 박지성\n 2) 차범근\n 3) 손웅정 ...',
  'combine_chat': 'Q.나이가 환갑이 넘은 직원은?  A.올해 예순(환갑)이 넘은 직원 목록

In [312]:
base_full_text_query = str(Text("combine_chat") % tokenize_query(user_query))
full_text_query = f"(~{base_full_text_query})"

In [313]:
query.set_filter(full_text_query)
query.query_string()

'(~@combine_chat:(더존비즈온의 | 30세이상 | 직원은?))=>[KNN 4 @chat_embeddings $vector AS vector_distance]'

In [317]:
from typing import Any, Dict, List
from redis.commands.search.aggregation import AggregateRequest, Desc

# Build the aggregation request
req = (
    AggregateRequest(query.query_string())
        .scorer("BM25")
        .add_scores()
        .apply(cosine_similarity="(2 - @vector_distance)/2", bm25_score="@__score")
        .apply(hybrid_score=f"0.3*@bm25_score + 0.7*@cosine_similarity")
        .load("chat_id", "chat", "answer", "combine_chat", "tags", "state")
        .sort_by(Desc("@hybrid_score"), max=4)
        .dialect(4)
)

# Run the query
res = index.aggregate(req, query_params={'vector': query._vector})
# Perform output parsing
[make_dict(row) for row in convert_bytes(res.rows)]

[{'vector_distance': '0.421030580997',
  '__score': '0.271592433443',
  'chat_id': 'a123126',
  'chat': '더존비즈온의 50세 이상 직원은?',
  'answer': '더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
  'combine_chat': 'Q.더존비즈온의 50세 이상 직원은?  A.더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
  'tags': 'tag8, tag10',
  'state': '1',
  'cosine_similarity': '0.789484709501',
  'bm25_score': '0.271592433443',
  'hybrid_score': '0.634117026684'},
 {'vector_distance': '0.437910437584',
  '__score': '0.271592433443',
  'chat_id': 'a123128',
  'chat': '더존비즈온의 60세 이상 직원은?',
  'answer': '더존비즈온의 50세 이상직원은 총 5명입니다. \n 1) 이순신\n 2) 세종대왕\n 3) 정약용 ...',
  'combine_chat': 'Q.더존비즈온의 60세 이상 직원은?  A.더존비즈온의 60세 이상직원은 총 3명입니다. \n 1) 카리나\n 2) 윈터\n 3) 차은우',
  'tags': 'tag5, tag99',
  'state': '0',
  'cosine_similarity': '0.781044781208',
  'bm25_score': '0.271592433443',
  'hybrid_score': '0.628209076879'},
 {'vector_distance': '0.562243103981',
  '__score': '0.0961538427004',
  'chat_id': 'a123129